## 3.3.4 LLM 기반의 RAG 시스템 구현

### 1. LLM 기반의 Reranker 클래스 구현

In [1]:
# Step1. 필요한 클래스 import
from llama_index.llms.openai import OpenAI # OpenAI API 사용을 위한 import
from llama_index.core.prompts import PromptTemplate # 프롬프트 템플릿
from llama_index.core import Document # 임의의 Document 객체를 만들기 위한 클래스 import

In [2]:
# Step2. LLM을 이용한 Reranker 클래스
class LLMReranker:
    def __init__(self, top_k:int = 2, model:str='gpt-4o', threshold:float=7.0):
        self.top_k = top_k # 관련성 점수 상위 k개 문서 반환
        self.threshold = threshold #  관련성 점수 기준(몇점 이상만 반환 1~10점)
        self.llm = OpenAI(model=model, temperature=0) # 일관적인 응답

    # OpenAI API를 활용하여 관련성 점수를 얻는 메서드
    def get_relevance_score(self, query:str, doc_text:str)->float:
        prompt_template=PromptTemplate(
                        template="""
                        1점부터 10점까지 [score]를 매겨서 아래 쿼리(query)와 문서(document)의 관련성을 평가하시오.
                        쿼리의 구체적인 맥락과 의도를 고려하여 정확히 평가하십시오.
                        question : {query}
                        document: {doc}

                        응답은 다음과 같이 반드시 [score]만 출력해주세요.
                        [score]는 소숫점 한자리까지 허용합니다.
                        출력시 다른 문자는 허용하지 않습니다.

                        응답 형태 : [score]만 출력
                        """,
                        template_vars = "query, doc" # template에서 취급 변수 지정
                        )
        prompt = prompt_template.format(query=query, doc=doc_text) # 탬플릿 포맷팅
        try:
            response = self.llm.complete(prompt) # LLM에 프롬프트 입력
            score = str(response)
            return float(score)
        except Exception as e: # 점수 취득에 에러가 발생한 경우 False 처리
            default_score = False
            return default_score
    
    # 문서 리스트를 입력받아 rerank 수행
    def rerank(self, query:str, documents: list[Document]): 
        scored_docs = []
        for doc in documents:
            score = self.get_relevance_score(query, doc.text) # 관련성 점수 취득 메서드 호출
            if score >= self.threshold: # 
                scored_docs.append((doc, score)) # (검색 노드, 점수)
        reranked_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True) # score 기준 내림차순 SORTING
        return reranked_docs[:self.top_k] # 상위 top_k개 문서를 반환

In [3]:
# 샘플 문서들
documents = [
    Document(text="인공지능의 발전과 미래 전망에 대한 연구"),
    Document(text="머신러닝 알고리즘의 성능 평가 방법론"),
    Document(text="딥러닝 모델의 학습 최적화 기법"),
    Document(text="교육 분야에서 인공지능의 활용 방안 연구"),
    Document(text="검색 증강 생성 시스템 연구 동향")
]

In [4]:
query = "인공지능의 미래는 어떨까요?"
reranker = LLMReranker(top_k=2, model='gpt-4o', threshold=0.0)
reranked_docs = reranker.rerank(query=query, documents=documents)

In [5]:
# 관련성 점수 상위 n개 순서대로 출력
for idx, (doc, score) in enumerate(reranked_docs, start=1):
    print(f"{idx}. {doc.text}, score : {score}")
    

1. 인공지능의 발전과 미래 전망에 대한 연구, score : 9.0
2. 교육 분야에서 인공지능의 활용 방안 연구, score : 3.0


### 2. 커스텀 검색기 작성

In [3]:
### Step3. Rerank 적용한 BM25+Custom Retriever 클래스 생성 
# Base query engine import
from llama_index.core.base.base_retriever import BaseRetriever

# BM25 -> LLM Rerank를 수행하는 Custom Retriever 구현
class LLMRerankRetriever(BaseRetriever):
    def __init__(self, base_retriever, reranker:LLMReranker):
        super().__init__()
        """
        :param index: 이미 생성된 Index (예:KeywordTableIndex 등)
        :param llm_reranker: 우리가 만든 LLMReranker 인스턴스
        :param num_candidates: 1차적으로 Index에서 몇 개를 가져올지
        """
        self.base_retriever = base_retriever
        self.reranker = reranker
    # retrieve 함수 coustomizing
    def _retrieve(self, query:str)->list[Document]:
        """
        1) 우선 BM25 Retriver를 통해 첫번째 문서 필터링
        2) LLMReranker로 문서를 재정렬한다.
        3) Query Engine에 반환할 문서 리스트를 반환한다.
        """
        # 1)Base Retriever
        initial_docs = self.base_retriever.retrieve(query)

        # 2) LLMReranker.rerank() -> (Document, score) 튜플
        #    만약 initial_docs가 Document 객체의 리스트라면 그대로 사용 가능
        reranked_docs_with_score = self.reranker.rerank(query, initial_docs)

        # 3) (Document, score) 튜플에서 Document만 추출
        #    Document + score가 모두 필요하다면 QueryEngine 쪽 custom 로직 필요
        final_docs = [doc_score_tuple[0] for doc_score_tuple in reranked_docs_with_score]
        return final_docs
        
    

In [4]:
### Step5. Rerank 적용한 BM25+Custom Retriever 클래스 생성 
# Base query engine import
from llama_index.core.base.base_retriever import BaseRetriever
from llama_index.core import QueryBundle

# BM25 -> 크로스인코더 Rerank를 수행하는 Custom Retriever 구현
class CustomRetriever(BaseRetriever):
    def __init__(self, base_retriever, reranker:LLMReranker):
        super().__init__()
        """
        :param base_retriever: 1st ranking 
        :param reranker: Cross Encoder 기반 리랭크 검색기
        """
        self.base_retriever = base_retriever 
        self.reranker = reranker
    # retrieve 함수 coustomizing
    def _retrieve(self, query_input:str)->list[Document]:
        # 1) QueryBundle이면 내부 query_str를 꺼냄
        if isinstance(query_input, QueryBundle): query = query_input.query_str
        else: query = query_input  # 그냥 문자열 혹은 그 외 케이스
            
        # 1) Base Retriever - 혼합검색
        try: initial_docs = self.base_retriever.retrieve(query)
        except: print('### Base Retrieve Error')
        
        # 2) rerank
        reranked_docs_with_score = self.reranker.rerank(query, initial_docs)
        
        # 3) (Document, score) 튜플에서 Document만 추출
        #    Document + score가 모두 필요하다면 QueryEngine 쪽 custom 로직 필요
        final_docs = [doc_score_tuple[0] for doc_score_tuple in reranked_docs_with_score]
        return final_docs

### 3. 필요한 문서 로드 -> 인덱스 로드로 대체

In [ ]:
# ### Step4. 데이터 로드
# from llama_index.core import SimpleDirectoryReader
# documents = SimpleDirectoryReader('./data').load_data()
# print(f'문서의 총 개수 : {len(documents)}')

### 4. BM25 검색기 객체 생성(1st ranker)

In [2]:
# ### Step5-1. 희소 검색을 위한 형태소 분석기 함수 작성
# from konlpy.tag import Okt
# okt = Okt() # Okt 형태소 분석기 초기화
# def tokenize_korean_text(text): # 문서 내용 토큰화 함수 정의
#     return okt.morphs(text)  # 한국어 형태소 기반 토큰화

In [3]:
# ### Step5-2. 청킹을 위한 Splitter 설정
# # 청킹 (chunking)
# from llama_index.core.node_parser import SentenceSplitter
# splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20)

In [6]:
# ### Step5-3. BM25 기반 인덱스 생성
# from llama_index.core.indices.keyword_table import KeywordTableIndex
# index = KeywordTableIndex.from_documents(documents=documents,
#                                          text_splitter=splitter,
#                                          extract_keyword=tokenize_korean_text,
#                                          max_keywords_per_node=10, # 각 청크에서 추출 된 상위 N개 키워드 선택
#                                          max_nodes_per_query=10, # 쿼리에 대해 반환 할 노드 수를 제한
#                                          show_progress=True # 진행 상황 확인
#                                         )

c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Parsing nodes:   0%|          | 0/23 [00:00<?, ?it/s]

Extracting keywords from nodes: 100%|██████████| 42/42 [00:53<00:00,  1.28s/it]


In [7]:
# ### Step5-4. Index 저장
# # 인덱스 로컬 경로 저장
# index.storage_context.persist(persist_dir="./index/ch03_3_4_keywrod_index_storage")

In [5]:
### Step5-5. 저장 된 인덱스 불러오기
from llama_index.core import StorageContext, load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="./index/ch03_keyword_index_storage") 
keyword_index = load_index_from_storage(storage_context)

In [6]:
### Step5-6. bm25 retriever 검색기 객체 생성
from llama_index.retrievers.bm25 import BM25Retriever
bm25_retriever = BM25Retriever.from_defaults(index=keyword_index,
    similarity_top_k=5, # 상위 5개 문서 반환
)

resource module not available on Windows


c:\Users\USER\OneDrive\Desktop\Project\llamaindex_practice\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from llama_index.core.response.notebook_utils import display_source_node
query = "중앙은행이 활용하고 있는 통화정책 수단을 알려주세요"
retrieved_nodes = bm25_retriever.retrieve(query)
retrieved_nodes = [result for result in retrieved_nodes if result.score > 0] # 0점인 것 제외
for node in retrieved_nodes:
    display_source_node(node, source_length=300)


**Node ID:** c744036b-6262-47aa-bc93-2c33b64559f8<br>**Similarity:** 3.826430082321167<br>**Text:** 14
경제금융용어 700선
정책당국이 취하는 제반 조치를 말한다. 이는 정책당국이 경제 전체의 총수요 수준을 
변동시킴으로써 경기 수위를 조절하는 데 초점을 맞추고 있다. 실제 운영에 있어서는 
정부지출과 세율을 조정하는 재정정책이 이용되거나 통화량과 금리 수준을 조절하는 
통화정책이 활용된다. 즉 경기가 정상수준을 큰 폭 밑도는 불황에 직면하게 될 경우 
정부는 재정지출을 늘리거나 조세를 줄이는 재정정책 수단을 동원한다. 한편 중앙은행은 
통화량을 늘리거나 금리를 내리는 정책수단을 활용한다. 이와는 반대로 경기가 지나치게 
...<br>

**Node ID:** d7461968-dfa0-4ba4-b336-8401dc4e47f0<br>**Similarity:** 2.9762978553771973<br>**Text:** 연관검색어 : 대안정기, 장기침체
공개시장운영
공개시장운영(open market operation)은 중앙은행이 금융시장에서 금융기관을 상대
로 국공채 등 증권을 매매하여 시중유동성이나 시장금리 수준에 영향을 미치는 통화정책
수단이다. 공개시장운영은 다른 통화정책수단(지급준비제도, 여수신제도 등)에 비해 
시기와 규모를 신축적으로 정할 수 있고 금융시장의 가격메커니즘에 따라 이루어지므로 
시장친화적인 데다 즉각적인 매매거래만으로 신속하게 정책을 시행할 수 있다는 장점이<br>

**Node ID:** d7081881-1989-4d8d-b0bf-92994573e21e<br>**Similarity:** 2.850802421569824<br>**Text:** 이처럼 통화공급경로를 주체별로 나누는 것은 통화량이 통화신용정책 이외에도 재정수
지 등 다양한 요인에 따라 크게 영향을 받기 때문에 통화정책 운용시에 이와 같은 
통화 등의 각 부문별 신용공급을 고려하기 위함이다. 중앙정부부문을 통한 통화공급은 
정부의 재정활동의 결과인 재정수지에 따라 그 규모가 결정된다. 민간부문을 통한 통화
공급은 민간신용이라고 하는데, 이는 금융기관이 기업과 가계에 대하여 대출을 해주거나 
기업이 발행한 유가증권을 금융기관이 매입하는 등의 형태로 이루어진다. 우리나라가 
물가안정목표제를 도입한 1998년...<br>

**Node ID:** 1b8cb7ab-ac11-4d8c-89a3-8580bcdad5c6<br>**Similarity:** 2.1005473136901855<br>**Text:** 26
경제금융용어 700선
삶의 질을 측정하기 위한 지표이다. 이 수치가 높을수록 실업자는 늘고 물가는 높아져 
한 나라의 국민이 느끼는 삶의 고통이 늘어남을 의미한다. 그러나 고통지수 (misery 
index)를 절대적인 것으로 생각해 나라 간에 단순 비교하기는 어려운 측면이 있는데 
이는 나라별로 소비자물가상승률과 실업률을 계산하는 기준이 다르고 빈부격차나 조사
대상에 따라서도 느끼는 고통의 정도가 상이할 수 있기 때문이다. 한편 고통지수가 
발표된 이후 이를 보완한 다양한 지표들이 개발되고 있는데, 1999년 미국 하버...<br>

**Node ID:** d5e0306e-9083-4dee-97bb-a4c1ecb823b7<br>**Similarity:** 1.9271259307861328<br>**Text:** 한편, 은행에서 취급하고 있는 금전신탁과 예금과의 차이점을 
살펴보면 운용방법에서는 금전신탁은 신탁법 및 신탁계약에서 정해진 것에 국한한 반면 
예금은 은행법 등에서 정한 범위 내에서 자유롭게 운용할 수 있다. 이익분배에 있어서는 
금전신탁은 실적배당, 예금은 확정이율을 원칙으로 한다 . 
기대인플레이션
기대인플레이션은 향후 물가상승률에 대한 경제주체의 주관적인 전망을 나타내는 
개념으로 물가안정을 추구하는 중앙은행이 관심을 기울이고 안정적으로 관리해야 하는 
기대인플레이션 ∙<br>

In [8]:
### Step8. reranker 객체 생성
llm_reranker = LLMReranker(top_k=2, model='gpt-4o', threshold=7.0)
reranked_docs = llm_reranker.rerank(query=query, documents=retrieved_nodes)

In [9]:
# 관련성 점수 상위 n개 순서대로 출력, 
for idx, (doc, score) in enumerate(reranked_docs, start=1):
    print(f"{idx}.Rerank nodes\n{doc.text}\nscore : {score}")

1.Rerank nodes
연관검색어 : 대안정기, 장기침체
공개시장운영
공개시장운영(open market operation)은 중앙은행이 금융시장에서 금융기관을 상대
로 국공채 등 증권을 매매하여 시중유동성이나 시장금리 수준에 영향을 미치는 통화정책
수단이다. 공개시장운영은 다른 통화정책수단(지급준비제도, 여수신제도 등)에 비해 
시기와 규모를 신축적으로 정할 수 있고 금융시장의 가격메커니즘에 따라 이루어지므로 
시장친화적인 데다 즉각적인 매매거래만으로 신속하게 정책을 시행할 수 있다는 장점이
score : 8.0
2.Rerank nodes
26
경제금융용어 700선
삶의 질을 측정하기 위한 지표이다. 이 수치가 높을수록 실업자는 늘고 물가는 높아져 
한 나라의 국민이 느끼는 삶의 고통이 늘어남을 의미한다. 그러나 고통지수 (misery 
index)를 절대적인 것으로 생각해 나라 간에 단순 비교하기는 어려운 측면이 있는데 
이는 나라별로 소비자물가상승률과 실업률을 계산하는 기준이 다르고 빈부격차나 조사
대상에 따라서도 느끼는 고통의 정도가 상이할 수 있기 때문이다. 한편 고통지수가 
발표된 이후 이를 보완한 다양한 지표들이 개발되고 있는데, 1999년 미국 하버드대 
배로(R. Barrow)교수는 오쿤의 고통지수에 국민소득증가율과 이자율을 감안한 ‘배로고
통지수’(BMI; Barrow Misery Index)를 발표한 바 있다 . 
 연관검색어 : 소비자물가지수(CPI), 실업률
골디락스경제
골디락스경제(Goldilocks economy)는 경기과열에 따른 인플레이션과 경기침체에 
따른 실업을 염려할 필요가 없는 최적 상태에 있는 건실한 경제를 가리킨다. 이는 영국의 
전래동화인 골디락스와 곰 세 마리(Goldilocks and the three bears)에 등장하는 금발머
리 소녀의 이름에서 유래하였다. 동화에 따르면 엄마 곰이 끓인 뜨거운 수프를 큰 접시와 
중간 접시 그리고 작은 접시에 담은 후 가족이 이를 식히기 위해 산책을 나갔는데 , 
이 때 

### 4. rerank 결과를 통한 LLM 응답 생성

In [10]:
### Step9. rerank_retriver 객체 생성
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
Settings.llm = OpenAI(model="gpt-4o", temperature=0.5,)

In [11]:
# 리랭크 검색기 객체 생성
rerank_retriever = CustomRetriever(base_retriever=bm25_retriever,
                                      reranker=llm_reranker,)

In [12]:
### Step10. query engine 생성 및 응답 확인
# Custom Retriever를 사용하기 위해선 RetrieverQueryEngine을 사용해야 함
from llama_index.core.query_engine import RetrieverQueryEngine
query_engine = RetrieverQueryEngine(retriever=rerank_retriever,) # query engine 구현

In [13]:
print(f"Query : {query}")
response = query_engine.query(query)
print(f"Response : {response}")

Query : 중앙은행이 활용하고 있는 통화정책 수단을 알려주세요
Response : 중앙은행이 활용하는 통화정책 수단 중 하나는 공개시장운영입니다. 이는 금융시장에서 국공채 등 증권을 매매하여 시중유동성이나 시장금리 수준에 영향을 미치는 방법입니다.


In [14]:
# 소스 문서 확인
for i, node_with_score in enumerate(response.source_nodes, start=1):
    print(f"{i}. \n{node_with_score.node.get_content()}")

1. 
연관검색어 : 대안정기, 장기침체
공개시장운영
공개시장운영(open market operation)은 중앙은행이 금융시장에서 금융기관을 상대
로 국공채 등 증권을 매매하여 시중유동성이나 시장금리 수준에 영향을 미치는 통화정책
수단이다. 공개시장운영은 다른 통화정책수단(지급준비제도, 여수신제도 등)에 비해 
시기와 규모를 신축적으로 정할 수 있고 금융시장의 가격메커니즘에 따라 이루어지므로 
시장친화적인 데다 즉각적인 매매거래만으로 신속하게 정책을 시행할 수 있다는 장점이
2. 
26
경제금융용어 700선
삶의 질을 측정하기 위한 지표이다. 이 수치가 높을수록 실업자는 늘고 물가는 높아져 
한 나라의 국민이 느끼는 삶의 고통이 늘어남을 의미한다. 그러나 고통지수 (misery 
index)를 절대적인 것으로 생각해 나라 간에 단순 비교하기는 어려운 측면이 있는데 
이는 나라별로 소비자물가상승률과 실업률을 계산하는 기준이 다르고 빈부격차나 조사
대상에 따라서도 느끼는 고통의 정도가 상이할 수 있기 때문이다. 한편 고통지수가 
발표된 이후 이를 보완한 다양한 지표들이 개발되고 있는데, 1999년 미국 하버드대 
배로(R. Barrow)교수는 오쿤의 고통지수에 국민소득증가율과 이자율을 감안한 ‘배로고
통지수’(BMI; Barrow Misery Index)를 발표한 바 있다 . 
 연관검색어 : 소비자물가지수(CPI), 실업률
골디락스경제
골디락스경제(Goldilocks economy)는 경기과열에 따른 인플레이션과 경기침체에 
따른 실업을 염려할 필요가 없는 최적 상태에 있는 건실한 경제를 가리킨다. 이는 영국의 
전래동화인 골디락스와 곰 세 마리(Goldilocks and the three bears)에 등장하는 금발머
리 소녀의 이름에서 유래하였다. 동화에 따르면 엄마 곰이 끓인 뜨거운 수프를 큰 접시와 
중간 접시 그리고 작은 접시에 담은 후 가족이 이를 식히기 위해 산책을 나갔는데 , 
이 때 집에 들어온 골디락스가 아기 곰 접시에 담긴 너무 뜨겁지도 않